# Feature Engineering

In [8]:
import pandas as pd
import numpy as np

In [9]:
pd.set_option("display.max_columns", None)

In [10]:
all_games = pd.read_csv("data/games_clean.csv", index_col="id")

In [99]:
all_games.shape

(49834, 17)

In [11]:
all_games.head()

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name
id,,,,,,,,,,,,,,,,,
47179,2019-01-30T00:00:00.000Z,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets
48751,2019-02-09T00:00:00.000Z,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers
48739,2019-02-08T00:00:00.000Z,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets
48740,2019-02-08T00:00:00.000Z,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers
48746,2019-02-08T00:00:00.000Z,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat


In [12]:
# Feature engineering

g = all_games.copy()

g["date"] = pd.to_datetime(all_games["date"]).dt.tz_localize(None)

# creating target variable
g["winner"] = np.where(g["home_team_score"] > g["visitor_team_score"], 1, 0)

# creating avg points by team across all years (not sure how useful this will be)
home_avg_pts_map = g[["home_team.id", "home_team_score"]].groupby("home_team.id").mean().squeeze()
g["home_team_avg_score_historical"] = g["home_team.id"].map(home_avg_pts_map).round(1)

visitor_avg_pts_map = g[["visitor_team.id", "visitor_team_score"]].groupby("visitor_team.id").mean().squeeze()
g["visitor_team_avg_score_historical"] = g["visitor_team.id"].map(visitor_avg_pts_map).round(1)

In [13]:
# creating an identifier for the team in a specific season
g["home_team_id_year"] = (g["home_team.id"].astype(str) + " " + g["season"].astype(str)).values
g["visitor_team_id_year"] = (g["visitor_team.id"].astype(str) + " " + g["season"].astype(str)).values

In [14]:
# creating average pts feature
# average pts scored per game in that season
# based on if they are home or away because there is a statistically significant difference
home_avg_score_map = g[["home_team_id_year", "home_team_score"]].groupby(["home_team_id_year"]).mean().squeeze()
visitor_avg_score_map = g[["visitor_team_id_year", "visitor_team_score"]].groupby(["visitor_team_id_year"]).mean().squeeze()

g["home_team_avg_score"] = g["home_team_id_year"].map(home_avg_score_map).round(1)
g["visitor_team_avg_score"] = g["visitor_team_id_year"].map(visitor_avg_score_map).round(1) 

In [15]:
# create an average difference in pts feature
# will calculate on average how much a team wins or loses by
# this will hopefully benefit lower scoring teams that also play defensively
avg_score_diff = g[["home_team_id_year", "home_team_avg_score", "visitor_team_avg_score"]].groupby("home_team_id_year").mean()
avg_score_diff = avg_score_diff["home_team_avg_score"] - avg_score_diff["visitor_team_avg_score"]
g["home_avg_score_diff"] = g["home_team_id_year"].map(avg_score_diff)

avg_score_diff = g[["visitor_team_id_year", "visitor_team_avg_score", "home_team_avg_score"]].groupby("visitor_team_id_year").mean()
avg_score_diff = avg_score_diff["visitor_team_avg_score"] - avg_score_diff["home_team_avg_score"]
g["visitor_avg_score_diff"] = g["visitor_team_id_year"].map(avg_score_diff)

In [16]:
g

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name,winner,home_team_avg_score_historical,visitor_team_avg_score_historical,home_team_id_year,visitor_team_id_year,home_team_avg_score,visitor_team_avg_score,home_avg_score_diff,visitor_avg_score_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,
47179,2019-01-30,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets,1,105.8,98.2,2 2018,4 2018,112.8,108.3,3.620000,-3.831707
48751,2019-02-09,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers,0,105.8,100.7,2 2018,13 2018,112.8,113.1,3.620000,0.581818
48739,2019-02-08,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets,1,103.3,104.2,23 2018,8 2018,117.9,108.2,8.725532,-4.670213
48740,2019-02-08,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers,1,103.4,98.3,30 2018,6 2018,116.4,103.8,7.429268,-8.419512
48746,2019-02-08,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat,1,105.5,96.0,26 2018,16 2018,114.9,105.4,5.129268,-6.670732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128052,2021-03-03,111,4,False,2020,Final,114,6,CLE,East,Central,Cleveland Cavaliers,12,IND,East,Central,Indiana Pacers,0,102.3,100.1,6 2020,12 2020,107.5,117.7,-4.341667,5.432432
128036,2021-03-01,124,4,False,2020,Final,130,22,ORL,East,Southeast,Orlando Magic,7,DAL,West,Southwest,Dallas Mavericks,0,101.9,101.5,22 2020,7 2020,105.4,113.8,-5.902778,1.257500
128001,2021-02-25,111,4,False,2020,Final,97,23,PHI,East,Atlantic,Philadelphia 76ers,7,DAL,West,Southwest,Dallas Mavericks,1,103.3,101.5,23 2020,7 2020,117.3,113.8,6.072093,1.257500


In [17]:
g.to_csv("data/games_with_features.csv")

In [18]:
feat_names = ["season", "winner", "home_team_avg_score", "visitor_team_avg_score"]
features = g.loc[:, feat_names]

features.to_csv("data/features.csv")

### Using stats

In [90]:
stats = pd.read_csv("data/all_stats_clean.csv", index_col="id")

In [91]:
# Convert game date to datetime
stats["game_date"] = pd.to_datetime(stats["game_date"]).dt.tz_localize(None)

# Convert string to timedelta
stats["min"] = [pd.Timedelta(minutes=int(time[0]), seconds=int(time[1])) for time in stats["min"].str.split(":").values]

# Create label
stats["winner"] = np.where(stats["home_team_score"].values > stats["away_team_score"].values, 1,0)
labels = stats[["game_id", "game_date", "season", "winner"]].groupby("game_id").first()

In [92]:
# split into 2 dataframes so that grouping data by game id doesn't
# group players of opposing teams
home_stats = stats[stats["player_team_id"].eq(stats["home_team_id"])]
away_stats = stats[stats["player_team_id"].eq(stats["away_team_id"])]

# define how to aggregate statistics when grouping player stats
agg_map = {"ast": "sum", 
           "blk": "sum", 
           "dreb": "sum", 
           "fg3_pct": "mean", 
           "fg3a": "sum", 
           "fg3m": "sum", 
           "fg_pct": "mean",
          "fga": "sum",
          "fgm": "sum",
          "ft_pct": "mean",
          "fta": "sum",
          "ftm": "sum",
          "min": "sum",
          "oreb": "sum",
          "pf": "sum",
          "pts": "sum",
          "reb": "sum",
          "stl": "sum",
          "turnover": "sum",
          "player_id": "first",
          "player_team_id": "first",
          "game_id": "first",
          "game_date": "first",
          "season": "first",
          "home_team_id": "first",
          "home_team_score": "first",
          "away_team_id": "first",
          "away_team_score": "first"}

# aggregate player stats to team stats
home_games = home_stats.groupby("game_id").agg(agg_map)
away_games = away_stats.groupby("game_id").agg(agg_map)

# player stats were aggregated to team stats, so player id doesn't make sense anymore
home_games.drop(["player_id", "player_team_id"], axis=1, inplace=True)
away_games.drop(["player_id", "player_team_id"], axis=1, inplace=True)

In [93]:
# sort rows by team and date to prepare for rolling average
home_games = home_games.sort_values(["home_team_id", "game_date"])
away_games = away_games.sort_values(["away_team_id", "game_date"])

In [94]:
stats_cols = ["ast","blk","dreb","fg3_pct","fg3a","fg3m","fg_pct","fga","fgm","ft_pct","fta","ftm","oreb",
              "pf","pts","reb","stl","turnover"]

rolling_stats_home = pd.DataFrame()
for team in home_games["home_team_id"].unique():
    games = home_games[home_games["home_team_id"].eq(team)]
    games_stats = games[stats_cols]
    rolling_stats_home = rolling_stats_home.append(games_stats.rolling(window=20).mean())
    rolling_stats_home = rolling_stats_home.shift()  # make the rolling average not include the current game
rolling_stats_home.dropna(inplace=True)


rolling_stats_away = pd.DataFrame()
for team in away_games["away_team_id"].unique():
    games = away_games[away_games["away_team_id"].eq(team)]
    games_stats = games[stats_cols]
    rolling_stats_away = rolling_stats_away.append(games_stats.rolling(window=20).mean())
    rolling_stats_away = rolling_stats_away.shift()  # make the rolling average not include the current game
rolling_stats_away.dropna(inplace=True)

# NOTE: adding columns to the start of the dataframe will mess up this code
# adding columns to the end will not
rolling_stats_diff = pd.DataFrame(index=rolling_stats.index)
home_cols = rolling_stats.columns[3:21]
away_cols = rolling_stats.columns[21:39]

In [96]:
##### This should be altered so that percentages are divided by each other instead of subtracted
for col in zip(stats_cols, home_cols, away_cols):
    rolling_stats_diff[col[0]] = rolling_stats[col[1]] - rolling_stats[col[2]]

In [97]:
rolling_stats_home.columns = ["home_" + col_name for col_name in rolling_stats_home.columns]
rolling_stats_away.columns = ["away_" + col_name for col_name in rolling_stats_away.columns]
rolling_stats_diff.columns = ["diff_" + col_name for col_name in rolling_stats_away.columns]

# putting it all together in 1 dataframe
rolling_stats = pd.merge(labels, rolling_stats_home, on="game_id")
rolling_stats = pd.merge(rolling_stats, rolling_stats_away, on="game_id")
rolling_stats = pd.merge(rolling_stats, rolling_stats_diff, on="game_id")

In [98]:
rolling_stats.to_csv("data/all_stats_feats.csv")

In [88]:
rolling_stats

,game_date,season,winner,home_ast,home_blk,home_dreb,home_fg3_pct,home_fg3a,home_fg3m,home_fg_pct,home_fga,home_fgm,home_ft_pct,home_fta,home_ftm,home_oreb,home_pf,home_pts,home_reb,home_stl,home_turnover,away_ast,away_blk,away_dreb,away_fg3_pct,away_fg3a,away_fg3m,away_fg_pct,away_fga,away_fgm,away_ft_pct,away_fta,away_ftm,away_oreb,away_pf,away_pts,away_reb,away_stl,away_turnover,diff_away_ast,diff_away_blk,diff_away_dreb,diff_away_fg3_pct,diff_away_fg3a,diff_away_fg3m,diff_away_fg_pct,diff_away_fga,diff_away_fgm,diff_away_ft_pct,diff_away_fta,diff_away_ftm,diff_away_oreb,diff_away_pf,diff_away_pts,diff_away_reb,diff_away_stl,diff_away_turnover
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,2018-10-19,2018,1,26.40,5.50,41.45,0.233344,39.00,12.95,0.406379,102.50,44.95,0.466162,28.65,21.05,11.80,22.65,123.90,53.25,7.40,14.75,23.60,6.20,34.55,0.211679,21.80,7.35,0.424561,85.95,39.45,0.420523,19.95,16.20,9.15,20.55,102.45,43.70,5.95,13.70,-0.35,-1.25,1.90,0.016533,12.35,4.00,-0.004522,3.65,-0.10,0.031138,5.95,2.60,1.65,-0.55,6.40,3.55,1.60,0.10
19,2018-10-19,2018,1,26.10,6.45,34.75,0.268088,34.35,12.95,0.468720,86.10,40.90,0.443043,21.35,16.95,8.70,20.95,111.70,43.45,7.65,13.35,24.40,4.50,40.55,0.272638,37.25,13.70,0.393084,99.50,44.40,0.451044,23.55,18.45,10.50,22.40,120.95,51.05,8.95,15.90,6.25,2.40,-0.10,0.004469,3.20,1.60,0.074468,2.80,3.35,0.013213,2.25,2.35,-0.10,1.85,10.65,-0.20,-0.25,-0.40
22,2018-10-19,2018,1,23.60,5.75,32.05,0.254139,24.00,8.40,0.446216,81.35,40.00,0.463828,25.30,20.30,7.45,20.90,108.70,39.50,8.00,12.45,21.75,4.30,32.50,0.236292,23.05,8.15,0.431402,86.65,41.20,0.444405,17.40,14.30,8.80,18.40,104.85,41.30,8.60,11.65,1.85,1.45,-0.45,0.017847,0.95,0.25,0.014814,-5.30,-1.20,0.019424,7.90,6.00,-1.35,2.50,3.85,-1.80,-0.60,0.80
24,2018-10-19,2018,1,22.40,5.70,33.45,0.225808,27.90,9.35,0.434261,86.40,37.80,0.459698,23.05,18.20,10.85,24.70,103.15,44.30,8.25,15.00,21.85,3.80,32.40,0.262395,26.85,9.80,0.403062,84.60,37.00,0.410959,20.25,15.50,10.35,19.50,99.30,42.75,7.70,15.85,-0.50,1.65,-0.80,-0.054324,-0.20,-0.90,0.020512,-4.25,-1.85,0.049093,2.15,2.05,0.20,3.10,-2.55,-0.60,-0.50,-1.60
26,2018-10-20,2018,1,23.45,4.60,34.75,0.291932,27.35,10.55,0.453450,89.15,43.25,0.432828,20.50,15.50,9.95,19.85,112.55,44.70,8.45,13.65,24.00,4.60,39.10,0.239598,34.50,10.95,0.423796,93.90,41.30,0.500859,27.50,21.25,10.15,22.50,114.80,49.25,7.95,17.55,1.10,0.30,-2.30,0.011969,-6.85,-1.20,0.034447,-1.35,3.90,-0.074475,-5.20,-4.35,-0.40,-1.30,2.25,-2.70,1.30,-3.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447185,2021-07-08,2020,1,27.25,4.30,35.05,29.717237,34.35,13.10,45.994763,88.35,43.95,40.082130,19.40,15.45,8.80,19.40,116.45,43.85,7.70,11.95,24.30,3.95,37.35,30.194355,34.75,12.90,42.793573,91.80,44.20,41.201587,20.35,16.55,9.25,18.20,117.85,46.60,8.90,13.75,2.95,0.35,-2.30,-0.477119,-0.40,0.20,3.201190,-3.45,-0.25,-1.119457,-0.95,-1.10,-0.45,1.20,-1.40,-2.75,-1.20,-1.80
447473,2021-07-11,2020,1,25.45,4.90,38.35,31.374339,38.30,15.40,42.120963,91.20,43.80,35.053765,21.15,15.85,10.60,15.50,118.85,48.95,7.55,13.75,25.45,4.75,31.60,29.985844,34.10,12.00,46.501079,88.10,42.40,46.126398,19.55,16.85,9.20,19.10,113.65,40.80,8.05,10.90,0.00,0.15,6.75,1.388496,4.20,3.40,-4.380116,3.10,1.40,-11.072633,1.60,-1.00,1.40,-3.60,5.20,8.15,-0.50,2.85
447761,2021-07-14,2020,1,24.55,4.95,37.75,31.553186,37.35,14.90,42.278463,89.20,42.75,35.354246,20.75,15.35,10.40,15.75,115.75,48.15,7.30,13.60,25.20,4.90,31.75,30.468408,34.00,12.30,46.765267,87.20,42.00,47.355031,20.20,17.50,9.20,19.05,113.80,40.95,8.05,11.20,-0.65,0.05,6.00,1.084778,3.35,2.60,-4.486804,2.00,0.75,-12.000785,0.55,-2.15,1.20,-3.30,1.95,7.20,-0.75,2.40
